# 1.라이브러리 임포트

In [1]:
import os
import torch
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.tools import Tool
from langchain.agents import initialize_agent, AgentType
from langchain.memory import ConversationBufferMemory
from langchain.document_loaders import JSONLoader
from dotenv import load_dotenv

# .env 파일 로드
load_dotenv()

# OpenAI API 키를 환경 변수에서 로드
openai_api_key = os.getenv("OPENAI_API_KEY")

# 2.JSON데이터 로드 및 처리

In [2]:
# JSON 문서 로드 및 처리
file_path = 'label_data.json'
jq_schema = """
.[] | {
    "filename": .filename,
    "original": .original,
    "id": .id,
    "date": .date,
    "conference_number": .conference_number,
    "question_number": .question_number,
    "meeting_name": .meeting_name,
    "generation_number": .generation_number,
    "committee_name": .committee_name,
    "meeting_number": .meeting_number,
    "session_number": .session_number,
    "agenda": .agenda,
    "law": .law,
    "qna_type": .qna_type,
    "context": .context,
    "context_learn": .context_learn,
    "context_summary": {
        "summary_q": .context_summary.summary_q,
        "summary_a": .context_summary.summary_a
    },
    "questioner": {
        "name": .questioner_name,
        "ID": .questioner_ID,
        "ISNI": .questioner_ISNI,
        "affiliation": .questioner_affiliation,
        "position": .questioner_position
    },
    "question": {
        "tag": .question.tag,
        "comment": .question.comment,
        "keyword": .question.keyword
    },
    "answerer": {
        "name": .answerer_name,
        "ID": .answerer_ID,
        "ISNI": .answerer_ISNI,
        "affiliation": .answerer_affiliation,
        "position": .answerer_position
    },
    "answer": {
        "tag": .answer.tag,
        "comment": .answer.comment,
        "keyword": .answer.keyword
    }
}
"""

# JSON 파일 로드
loader = JSONLoader(file_path, jq_schema=jq_schema, text_content=False)
documents = loader.load()

# 3.문서 분할 및 임베딩 생성

In [3]:
# 텍스트 분할기 설정
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
split_docs = text_splitter.split_documents(documents)

# CUDA 사용 설정
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 임베딩 모델 설정 (GPU 사용)
embedding_model_name = 'sentence-transformers/all-MiniLM-L6-v2'
embedding_model = HuggingFaceEmbeddings(
    model_name=embedding_model_name,
    model_kwargs={'device': device}
)

# FAISS 인덱스 설정
index_path = 'faiss_index'

# VectorStore 생성 또는 로드
if os.path.exists(index_path):
    print("저장된 FAISS 인덱스를 로드합니다...")
    vectorstore = FAISS.load_local(
        index_path,
        embeddings=embedding_model,
        allow_dangerous_deserialization=True
    )
else:
    print("FAISS 인덱스를 생성합니다...")
    vectorstore = FAISS.from_documents(split_docs, embedding_model)
    vectorstore.save_local(index_path)


/tmp/ipykernel_121698/2611998689.py:10: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(
/home/elicer/.local/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


FAISS 인덱스를 생성합니다...


# 4.Retriever 및 도구 정의

In [4]:
# Retriever 생성
retriever = vectorstore.as_retriever(search_kwargs={"k": 6})

# 도구 함수 정의
def json_search_tool(input_text):
    docs = retriever.get_relevant_documents(input_text)
    if docs:
        summaries = [doc.page_content for doc in docs]
        return '\n\n'.join(summaries)
    else:
        return "해당하는 정보를 찾을 수 없습니다."

# 도구 정의
json_search_tool = Tool(
    name="JSONSearch",
    func=json_search_tool,
    description=(
        "Use this tool to search for information about a specific person or issue from the JSON document. "
        "Provides a summary of fields such as conference number, meeting name, generation number, committee name, agenda, law, Q&A type, context, learning context, "
        "context summary, questioner, and question."
    )
)


In [5]:
tools = [json_search_tool]


# 6. 도구 목록 생성

In [6]:
# 도구 목록 생성
tools = [json_search_tool]

# LLM 정의
llm = ChatOpenAI(model_name="gpt-4", temperature=0, openai_api_key=openai_api_key)

# 메모리 설정
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

/tmp/ipykernel_121698/3727535949.py:5: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name="gpt-4", temperature=0, openai_api_key=openai_api_key)
/tmp/ipykernel_121698/3727535949.py:8: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


In [7]:
# 에이전트 초기화
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
    verbose=True,
    memory=memory,
)


/tmp/ipykernel_121698/2170808627.py:2: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 1.0. Use :meth:`~Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc.` instead.
  agent = initialize_agent(


In [8]:
# 시스템 메시지 설정
agent.agent.llm_chain.prompt.messages[0].prompt.template = ("Be sure to answer in Korean"
    "You are an AI assistant that helps users find information about individuals or issues from the National Assembly JSON document. "
    "When a user asks about a specific person or issue, you should use the JSONSearch tool to find meeting information where that person is mentioned. "
    "Provide a summarized response including fields such as conference number, meeting name, generation number, committee name, agenda, law, Q&A type, context, learning context, context summary, questioner, and question. "
    "If you cannot find the information in the JSON document, politely inform the user that the information is not available. "
    "Do not mention the use of tools; just provide the necessary information."
)


In [9]:
def chat_with_agent(user_input):
    response = agent.run(input=user_input)
    return response


In [10]:
# 예시 질문
user_input = "김영삼 대통령이 언급된 회의에 대한 정보를 알려주세요.반드시 안건 이름과 날짜 어떤 회의 였는지 몇대 국회였는지 알려주세요. 질문자와 답변자에 대한 정보도 알려주세요."
response = chat_with_agent(user_input)
print("Assistant:", response)

# 추가 질문
user_input = "그 회의에서 주요 안건은 무엇이었나요?"
response = chat_with_agent(user_input)
print("Assistant:", response)


/tmp/ipykernel_121698/1973639900.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = agent.run(input=user_input)




> Entering new AgentExecutor chain...
```json
{
    "action": "JSONSearch",
    "action_input": "김영삼"
}
```
Observation: reverse these decisions for the benefit of Seoul citizens and public housing integrity.", "context_summary": {"summary_q": "\ubc15\uc6d0\uc21c \uc2dc\uc7a5\ub2d8 \uc2dc\uc808\uc5d0 \ud6c4\ubd84\uc591 \uacf5\uc815\ub960\uc744 \ub0ae\ucd94\uace0, \ubd84\uc591\uc6d0\uac00 \uacf5\uac1c \ud56d\ubaa9\uc744 \uc904\uc774\ub294 \ub4f1\uc758 \uacb0\uc815\uc774 \uc11c\ubbfc\uc758 \uc8fc\ud0dd \ubb38\uc81c\ub97c \uc545\ud654\uc2dc\ucf30\ub2e4\ub294 \uc9c0\uc801\uc774 \uc788\uc2b5\ub2c8\ub2e4. \uc774\uc5d0 \ub300\ud55c \uc785\uc7a5\uc774 \uc5b4\ub5bb\uc2b5\ub2c8\uae4c?", "summary_a": "\ubd84\uc591\uc6d0\uac00 \uacf5\uac1c \ub4f1\uc758 \uacb0\uc815\uc740 \ub2f9\uc2dc\uc758 \uc5ec\ub7ec \uaddc\uc815 \ubcc0\ud654\ub85c \uc778\ud55c \uac83\uc73c\ub85c \ud310\ub2e8\ub429\ub2c8\ub2e4. \uc774\ub7ec\ud55c \uacb0\uc815\uc774 \uc8fc\ud0dd \ubb38\uc81c \uc545\ud654\uc5d0 \uae30\uc5ec\ud55

/tmp/ipykernel_121698/4141912307.py:6: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents(input_text)


```json
{
    "action": "Final Answer",
    "action_input": "김영삼 대통령에 대한 언급이 있는 회의 정보는 다음과 같습니다. 회의 번호는 109이며, 회의 이름은 '경남도민일보 관련 녹취록 공개'입니다. 이 회의는 20대 국회에서 진행되었습니다. 안건 이름은 '후분양 공정률 낮추기, 분양원가 공개 항목 줄이기'이며, 질문자는 윤상직 위원이었습니다. 그는 '그러네요. 경남도민일보가 관련 녹취록을 공개했습니다. 그 녹취록에는 조덕이 민주당 인사들에게 룸살롱에서 돈을 전달하는 정황이 그대로 녹음되어 있습니다.'라고 질문하였습니다. 답변자는 허원 부위원장이었으며, 그의 답변은 '해당 사건은 현재 통영지검에서 수사 중이며, 수사 과정에서 필요한 모든 조치를 취하고 신속하게 진행하도록 하겠습니다.'였습니다."
}
```

> Finished chain.
Assistant: 김영삼 대통령에 대한 언급이 있는 회의 정보는 다음과 같습니다. 회의 번호는 109이며, 회의 이름은 '경남도민일보 관련 녹취록 공개'입니다. 이 회의는 20대 국회에서 진행되었습니다. 안건 이름은 '후분양 공정률 낮추기, 분양원가 공개 항목 줄이기'이며, 질문자는 윤상직 위원이었습니다. 그는 '그러네요. 경남도민일보가 관련 녹취록을 공개했습니다. 그 녹취록에는 조덕이 민주당 인사들에게 룸살롱에서 돈을 전달하는 정황이 그대로 녹음되어 있습니다.'라고 질문하였습니다. 답변자는 허원 부위원장이었으며, 그의 답변은 '해당 사건은 현재 통영지검에서 수사 중이며, 수사 과정에서 필요한 모든 조치를 취하고 신속하게 진행하도록 하겠습니다.'였습니다.


> Entering new AgentExecutor chain...
```json
{
    "action": "JSONSearch",
    "action_input": "109번 회의 주요 안건"
}
```
Observation: reverse these decisions for the benefit of Seou